# Implementing GPT 2 from scratch

# A Dummy GPT_2 model

In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "dropout_rate": 0.1,
    "qkv_bias": False
}

In [1]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_embedding = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.positional_embedding = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.dropout_emb = nn.Dropout(cfg["dropout_rate"])

        self.transformer_blocks = nn.Sequential(
          *[DummyTransformerBlock(cfg)  for _ in range(cfg["n_layers"])])

        self.final_layer_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)


    def forward(self, inputs):
        batch_size, seq_len = inputs.shape
        token_embeds = self.token_embedding(inputs)
        positional_embeds = self.positional_embedding(torch.arange(seq_len, device=inputs.device))
        x = token_embeds + positional_embeds
        x = self.dropout_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_layer_norm(x)
        logits = self.out_head(x)

        return logits
        

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):

        return x
        
class DummyLayerNorm(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):

        return x    


In [13]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

text_1 = "Every effort moves you"
text_2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(text_1)))
batch.append(torch.tensor(tokenizer.encode(text_2)))

batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

In [19]:
torch.manual_seed(123)
model = DummyGPTModel(cfg=GPT_CONFIG_124M)
logits = model(batch)
logits.shape

torch.Size([2, 4, 50257])

# Layer Normalization

In [20]:
torch.manual_seed(123)

batch_example = torch.randn(2, 5) #token with 5 embed_dim
batch_example

tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969],
        [ 0.2093, -0.9724, -0.7550,  0.3239, -0.1085]])

In [21]:
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
output = layer(batch_example)
output

tensor([[0.2260, 0.3470, 0.0000, 0.2216, 0.0000, 0.0000],
        [0.2133, 0.2394, 0.0000, 0.5198, 0.3297, 0.0000]],
       grad_fn=<ReluBackward0>)

In [26]:
mean = output.mean(dim=-1, keepdim=True)
mean

tensor([[0.1324],
        [0.2170]], grad_fn=<MeanBackward1>)

In [33]:
variance = output.var(dim=-1, keepdim=True)
variance

tensor([[0.0231],
        [0.0398]], grad_fn=<VarBackward0>)

In [34]:
normalized = ((output - mean) / torch.sqrt(variance))
    
normalized.mean(dim=-1, keepdim=True)

tensor([[    -0.0000],
        [     0.0000]], grad_fn=<MeanBackward1>)

In [29]:
torch.set_printoptions(sci_mode=False)

In [35]:
normalized.var(dim=-1, keepdim=True)

tensor([[1.0000],
        [1.0000]], grad_fn=<VarBackward0>)

In [39]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var =  x.var(dim=-1, keepdim=True)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)

        return self.scale * norm_x + self.shift

In [40]:
layer_norm = LayerNorm(6)
norm_output = layer_norm(output)
norm_output

tensor([[ 0.6157,  1.4123, -0.8717,  0.5871, -0.8717, -0.8717],
        [-0.0189,  0.1121, -1.0875,  1.5171,  0.5647, -1.0875]],
       grad_fn=<AddBackward0>)

# Feed Forward Network 

In [41]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):

        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.44715 * torch.pow(x, 3))
        ))

In [55]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"])
        )

    def forward(self, x):  
        return self.layers(x)

In [47]:
ffn = FeedForwardNetwork(cfg=GPT_CONFIG_124M)

x = torch.rand(2, 3, 768)
ffn(x).shape

torch.Size([3, 768])

In [62]:
ffn.layers

Sequential(
  (0): Linear(in_features=768, out_features=3072, bias=True)
  (1): GELU()
  (2): Linear(in_features=3072, out_features=768, bias=True)
)

# Transformer Block

In [74]:
from previous_chapters import MultiHeadAttention

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.mha = MultiHeadAttention(
            d_in = cfg["emb_dim"],
            d_out = cfg["emb_dim"],
            context_length = cfg["context_length"],
            dropout = cfg["dropout_rate"],
            num_heads = cfg["n_heads"],
            qkv_bias = cfg["qkv_bias"]
        )
        self.ffn = FeedForwardNetwork(cfg)
        self.layer_norm_1 = LayerNorm(cfg["emb_dim"])
        self.layer_norm_2 = LayerNorm(cfg["emb_dim"])
        self.dropout_shortcut = nn.Dropout(cfg["dropout_rate"])

    def forward(self, x):
        shortcut = x
        x = self.layer_norm_1(x)
        x = self.mha(x)
        x = self.dropout_shortcut(x)
        x = x + shortcut

        shorcut = x
        x = self.layer_norm_2(x)
        x = self.ffn(x)
        x = self.dropout_shortcut(x)
        x = x + shortcut

        return x
        

In [75]:
torch.manual_seed(123)

x = torch.randn(2, 4, 768)
tf_block = TransformerBlock(cfg=GPT_CONFIG_124M)

output = tf_block(x)
output.shape

torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])


torch.Size([2, 4, 768])

# Original GPT 2 model

In [76]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

batch = []

text_1 = "Every effort moves you"
text_2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(text_1)))
batch.append(torch.tensor(tokenizer.encode(text_2)))

batch = torch.stack(batch, dim=0)
batch

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

In [77]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.token_embedding = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.positional_embedding = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.dropout_emb = nn.Dropout(cfg["dropout_rate"])

        self.transformer_blocks = nn.Sequential(
          *[TransformerBlock(cfg)  for _ in range(cfg["n_layers"])])

        self.final_layer_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)


    def forward(self, inputs):
        batch_size, seq_len = inputs.shape
        token_embeds = self.token_embedding(inputs)
        positional_embeds = self.positional_embedding(torch.arange(seq_len, device=inputs.device))
        x = token_embeds + positional_embeds
        x = self.dropout_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_layer_norm(x)
        logits = self.out_head(x)

        return logits

In [78]:
torch.manual_seed(123)

model = GPTModel(cfg=GPT_CONFIG_124M)
output = model(batch)
output.shape

torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4])
torch.Size([2, 4, 12, 64])
torch.Size([2, 4, 768])
torch.Size([2, 4, 768])
torch.Size([2, 12, 4, 64])
torch.Size([2, 12, 4, 4])
torch.Size([2, 12, 4, 4

torch.Size([2, 4, 50257])

In [79]:
batch.numel()

8

In [81]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,}") #weight sharing in original gpt model emb layer and out_head layer

163,009,536


In [82]:
total_params - model.token_embedding.weight.numel()

124412160

# Generating Text

In [99]:
start_context = "Hello, I am"

encoded = tokenizer.encode(start_context)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
encoded_tensor.shape

torch.Size([1, 4])

In [100]:
def generate_text(model, idx, max_new_tokens, context_size):
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]

        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)

    return idx
        
        

In [101]:
output = generate_text(
    model=model,
    idx=encoded_tensor,
    max_new_tokens=6,
    context_size=GPT_CONFIG_124M["context_length"])


torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 12, 64])
torch.Size([1, 4, 768])
torch.Size([1, 4, 768])
torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 12, 64])
torch.Size([1, 4, 768])
torch.Size([1, 4, 768])
torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 12, 64])
torch.Size([1, 4, 768])
torch.Size([1, 4, 768])
torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 12, 64])
torch.Size([1, 4, 768])
torch.Size([1, 4, 768])
torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 12, 64])
torch.Size([1, 4, 768])
torch.Size([1, 4, 768])
torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4])
torch.Size([1, 4, 12, 64])
torch.Size([1, 4, 768])
torch.Size([1, 4, 768])
torch.Size([1, 12, 4, 64])
torch.Size([1, 12, 4, 4])
torch.Size([1, 12, 4, 4

In [102]:
output

tensor([[15496,    11,   314,   716,  5466, 43474, 38040, 37516, 10088, 42030]])

In [103]:
tokenizer.decode(output.squeeze(0).tolist())

'Hello, I am sale Discount promul eighty operatorfoundland'